<a href="https://colab.research.google.com/github/rachanrv/Twitter-Mining-with-GCP/blob/master/Audio_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recording stuff

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [ ]:
%pip install SpeechRecognition

In [ ]:
# all imports
from io import BytesIO
from base64 import b64decode
from google.colab import output
from IPython.display import Javascript

In [ ]:
RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=3):
  print("Speak Now...")
  display(Javascript(RECORD))
  sec += 1
  s = output.eval_js('record(%d)' % (sec*1000))
  print("Done Recording !")
  b = b64decode(s.split(',')[1])
  return b #byte stream

In [ ]:
audio = record(5) 

Speak Now...


<IPython.core.display.Javascript object>

Done Recording !


In [ ]:
import IPython.display as ipd
ipd.display(ipd.Audio(audio))

In [ ]:
with open('/content/drive/My Drive/Python Assignments/Project/myfile.wav', mode='bx') as f:
    f.write(audio)

OSError: ignored

# DOESN'T WORK

In [ ]:
#import library
import speech_recognition as sr

# Initialize recognizer class (for recognizing the speech)
r = sr.Recognizer()

# Reading Audio file as source
# listening the audio file and store in audio_text variable

with sr.AudioFile('/content/drive/My Drive/Python Assignments/Project/myfile.wav') as source:
    
    audio_text = r.listen(source)
    
# recoginize_() method will throw a request error if the API is unreachable, hence using exception handling
    try:
        
        # using google speech recognition
        text = r.recognize_google(audio_text)
        print('Converting audio transcripts into text ...')
        print(text)
     
    except:
         print('Sorry.. run again...')

# CODE for using Google Cloud Platform

In [ ]:
from google.colab import drive
import os
!ls /content/gdrive/’My Drive’/’Colab Notebooks’/tempimport os
os.environ[“GOOGLE_APPLICATION_CREDENTIALS”]=”/content/drive/My Drive/Colab Notebooks/speech-to-text-api-key.json” #ensure the path is set correctly
!echo $GOOGLE_APPLICATION_CREDENTIALS

In [ ]:
%pip install --upgrade google-cloud-speech

In [ ]:
%pip install pydub
%pip install sox
!apt -qq install -y sox

In [ ]:
# STEP 1: Convert .mp3 to .wav file
# STEP 2: Use subprocess + sox to convert .wav file into MONO .wav file, make sure sample_rate_hertz = 44100import subprocess
import os
from pydub import AudioSegment# dir
dirname = '/content/gdrive/My Drive/Colab Notebooks/temp/tbd_convert/'# function that takes a file and converts to the mono version
def convert_audio(filename):# files
  filenamesplit = os.path.splitext(filename)[0]
  filenamewav = filenamesplit + '.wav'
  print(filenamewav)
  newfilename = "mono_" + filenamewav# convert mp3 to wav                                                            
  sound = AudioSegment.from_mp3(dirname + filename)
  sound.export(dirname + filenamewav, format="wav")# convert wav to wav mono
  command2 = ['sox',dirname + filenamewav,'-c','1',dirname + newfilename]
  subprocess.Popen(command2)for filename in os.listdir(dirname):
  print(filename)
  print("Starting...")
  convert_audio(filename)
  print("Completed!!")

In [ ]:
# STEP 3: Copies all files from temp/tbd_convert folder into Google Cloud Storage bucketfrom google.colab import auth
from google.cloud import storageauth.authenticate_user()
project_id = 'project-260116'
!gcloud config set project {project_id}
!gsutil lsbucket_name = 'sp2tx'
!gsutil -m cp -r /content/gdrive/My\ Drive/Colab\ Notebooks/temp/tbd_convert/* gs://{bucket_name}/

In [ ]:
%pip install --upgrade google-cloud-storage

In [ ]:
# TEST
from google.cloud import storageproject_id = 'project-260116'
bucket_name = 'sp2tx'def list_blobs(bucket_name):
    """Lists all the blobs in the bucket."""# Note: Client.list_blobs requires at least package version 1.17.0.
    storage_client = storage.Client(project_id)
    blobs = storage_client.list_blobs(bucket_name)for blob in blobs:
      print(blob.name)list_blobs(bucket_name)

In [ ]:
# STEP 4: Use Google Speech to Text to convert into transcriptions from google.cloud import speech_v1
from google.cloud.speech_v1 import enums
from google.colab import auth
from google.cloud import storageproject_id = 'project-260116'
bucket_name = 'sp2tx'def sample_long_running_recognize(storage_uri):
    """
    Transcribe long audio file from Cloud Storage using asynchronous speech
    recognitionArgs:
      storage_uri URI for audio file in Cloud Storage, e.g. gs://[BUCKET]/[FILE]
    """client = speech_v1.SpeechClient()# Sample rate in Hertz of the audio data sent
    sample_rate_hertz = 44100# The language of the supplied audio
    language_code = "en-US"# Encoding of audio data sent. This sample sets this explicitly.
    # This field is optional for FLAC and WAV audio formats.
    encoding = enums.RecognitionConfig.AudioEncoding.LINEAR16
    config = {
        "sample_rate_hertz": sample_rate_hertz,
        "language_code": language_code,
        "encoding": encoding,
    }
    audio = {"uri": storage_uri}operation = client.long_running_recognize(config, audio)print(u"Waiting for operation to complete...")
    response = operation.result()for result in response.results:
        # First alternative is the most probable result
        alternative = result.alternatives[0]
        print(u"Transcript: {}".format(alternative.transcript))def blob_bucket_transcribe(bucket_name):
    """Lists all the blobs in the bucket."""# Note: Client.list_blobs requires at least package version 1.17.0.
    storage_client = storage.Client(project_id)
    blobs = storage_client.list_blobs(bucket_name)storage_prefix = 'gs://' + bucket_name + '/'for blob in blobs:
      print(blob.name)
      storage_uri = blob.name
      sample_long_running_recognize(storage_prefix + storage_uri)
      
#storage_uri = 'gs://sp2tx/mono_mike_liu_01.wav'
#sample_long_running_recognize(storage_uri)blob_bucket_transcribe(bucket_name)